In [2]:
pip install slither-analyzer==0.10.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of web3 to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of web3 to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of eth-account to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of eth-rlp to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 19.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━

In [7]:
import shutil
import os

src_dir = "/kaggle/input/resourcee/graph/.solc-select/artifacts"
dst_dir = "/kaggle/working/artifacts"

if os.path.exists(dst_dir):
    shutil.rmtree(dst_dir)
shutil.copytree(src_dir, dst_dir)




'/kaggle/working/artifacts'

In [8]:
!chmod -R +x /kaggle/working/artifacts


In [11]:
import os
import json

from os.path import join
from shutil import copy
from copy import deepcopy
from re import L
from typing import Pattern
from tqdm import tqdm
import re

import networkx as nx
from slither.slither import Slither
from slither.core.cfg.node import NodeType


pattern =  re.compile(r'\d.\d.\d+')

def get_solc_version(source):
    PATTERN = re.compile(r"pragma solidity\s*(?:\^|>=|<=)?\s*(\d+\.\d+\.\d+)")
    solc_select ="/kaggle/working/artifacts"
    solc_version = [v.split('-')[-1] for v in os.listdir(solc_select)]
    with open(os.path.join(source), encoding="utf8") as file_desc:
        buf = file_desc.read()
    version = PATTERN.findall(buf)
    version = '0.4.25' if len(version) == 0 else version[0]
    if version == '0.4.26':
        version = '0.4.26'
    elif version == '0.4.17':
        version = '0.4.17'
    elif version.startswith('0.4.'):
        version = '0.4.25'
    elif version.startswith('0.5.'):
        version = '0.5.16'
    elif version == '0.8.0':
        version = '0.8.6'
          

    if version not in solc_version:
        if version.startswith('0.4.'):
            print("asdasdas")
        elif version.startswith('0.5.'):
            solc_path = os.path.join(solc_select, 'solc-' + '0.5.11', 'solc-' + '0.5.11')
        else:
            solc_path = os.path.join(solc_select, 'solc-' + '0.8.6', 'solc-' + '0.8.6')
    else:
        solc_path = os.path.join(solc_select, 'solc-' + version, 'solc-' + version)
    return solc_path
def get_node_info(node, list_vulnerabilities_info_in_sc):
    node_label = "Node Type: {}\n".format(str(node.type))
    node_type = str(node.type)
    if node.expression:
        node_label += "\nEXPRESSION:\n{}\n".format(node.expression)
        node_expression = str(node.expression)
    else:
        node_expression = None
    if node.irs:
        node_label += "\nIRs:\n" + "\n".join([str(ir) for ir in node.irs])
        node_irs = "\n".join([str(ir) for ir in node.irs])
    else:
        node_irs = None
 
    node_source_code_lines = node.source_mapping.lines
    node_info_vulnerabilities = get_vulnerabilities_of_node_by_source_code_line(node_source_code_lines, list_vulnerabilities_info_in_sc)
    return node_label, node_type, node_expression, node_irs, node_info_vulnerabilities, node_source_code_lines

def get_vulnerabilities(file_name_sc, vulnerabilities):
    list_vulnerability_in_sc = None
    if vulnerabilities is not None:
        for vul_item in vulnerabilities:
            #if file_name_sc == vul_item['name']:
                #list_vulnerability_in_sc = vul_item['vulnerabilities']
            if file_name_sc == vul_item['name']:
                list_vulnerability_in_sc = vul_item['vulnerabilities']
                source = vul_item.get('source')
    return list_vulnerability_in_sc, source

def get_vulnerabilities_of_node_by_source_code_line(source_code_lines, list_vul_info_sc):
    if list_vul_info_sc is not None:
        list_vulnerability = []
        for vul_info_sc in list_vul_info_sc:
            vulnerabilities_lines = vul_info_sc['lines']
            #print(vulnerabilities_lines)
            # for source_code_line in source_code_lines:
            #     for vulnerabilities_line in vulnerabilities_lines:
            #         if source_code_line == vulnerabilities_line:
            #             list_vulnerability.append(vul_info_sc)
            interset_lines = set(vulnerabilities_lines).intersection(set(source_code_lines))
            #print("SCL : :: : :: : : : ",source_code_lines)
            #print("VUL : :: : :: : : : ",vulnerabilities_lines)
            #print(interset_lines)
            if len(interset_lines) > 0:
                list_vulnerability.append(vul_info_sc)
        if len(list_vulnerability) == 0:
            return "None"
        else:
            return list_vulnerability


    else:
        list_vulnerability = None
    
    if list_vulnerability is None or len(list_vulnerability) == 0:
        node_info_vulnerabilities = None
    else:
        node_info_vulnerabilities = list_vulnerability

    return node_info_vulnerabilities

def compress_full_smart_contracts(smart_contracts, input_graph, output, vulnerabilities=None):
    full_graph = None
    if input_graph is not None:
        full_graph = nx.read_gpickle(input_graph)
    compiled_ok_path = "/kaggle/working/out/compiled_success.txt"
    compiled_fail_path = "/kaggle/working/out/compiled_failed.txt"

    compiled_ok = []
    compiled_fail = []
    count = 0
    solc_remaps = "@openzeppelin=D:/UIT/KLTN/graph/github/dappscan_github/node_modules/@openzeppelin" # kệ bỏ qua
    base_dir = "/kaggle/input/resourcee/graph/exported_sols"

    for sc in tqdm(smart_contracts):
        sc = sc.replace("home/kali/KLTN/", "")
        file_name = os.path.basename(sc)
        sc = os.path.join(base_dir, file_name)
        print()

        sc_version = get_solc_version(sc)
        
        print(sc_version)

        #solc_compiler = "/home/kali/.solc-select/artifacts/solc-{sc_version}/solc-{sc_version}"
        #if not os.path.exists(solc_compiler):
            #solc_compiler = "/home/kali/.solc-select/artifacts/solc-0.4.25/solc-0.4.25"
        file_name_sc = sc.split('/')[-1:][0]
        bug_type = sc.split('/')[-2]
        try:
            slither = Slither(sc, solc=sc_version, solc_remaps=[solc_remaps] )
            count += 1
            print(count)
            compiled_ok.append(sc)
        except Exception as e:
            print(sc)
            print("Error",e)
            compiled_fail.append(sc)
            continue	 
        
        list_vul_info_sc, source = get_vulnerabilities(file_name_sc, vulnerabilities)
        print(list_vul_info_sc)





        merge_contract_graph = None
        for contract in slither.contracts:
            merged_graph = None
            for idx, function in enumerate(contract.functions + contract.modifiers):  

                nx_g = nx.MultiDiGraph()
                for nidx, node in enumerate(function.nodes):           
                    node_label, node_type, node_expression, node_irs, node_info_vulnerabilities, node_source_code_lines = get_node_info(node, list_vul_info_sc)
                    

                    
                  
                
                    
                    nx_g.add_node(node.node_id, label=node_label,
                                  node_type=node_type, node_expression=node_expression, node_irs=node_irs,
                                  node_info_vulnerabilities=node_info_vulnerabilities,
                                  node_source_code_lines=node_source_code_lines,
                                  function_fullname=function.full_name, contract_name=contract.name, source_file=os.path.join(source, file_name_sc))
                    
                    if node.type in [NodeType.IF, NodeType.IFLOOP]:
                        true_node = node.son_true
                        if true_node:
                            if true_node.node_id not in nx_g.nodes():
                                node_label, node_type, node_expression, node_irs, node_info_vulnerabilities, node_source_code_lines = get_node_info(true_node, list_vul_info_sc)
                                nx_g.add_node(true_node.node_id, label=node_label,
                                              node_type=node_type, node_expression=node_expression, node_irs=node_irs,
                                              node_info_vulnerabilities=node_info_vulnerabilities,
                                              node_source_code_lines=node_source_code_lines,
                                              function_fullname=function.full_name, contract_name=contract.name, source_file=os.path.join(source, file_name_sc))
                            nx_g.add_edge(node.node_id, true_node.node_id, edge_type='if_true', label='True')
                        
                        
                        false_node = node.son_false
                        if false_node:
                            if false_node.node_id not in nx_g.nodes():
                                node_label, node_type, node_expression, node_irs, node_info_vulnerabilities, node_source_code_lines = get_node_info(false_node, list_vul_info_sc)
                                nx_g.add_node(false_node.node_id, label=node_label,
                                              node_type=node_type, node_expression=node_expression, node_irs=node_irs,
                                              node_info_vulnerabilities=node_info_vulnerabilities,
                                              node_source_code_lines=node_source_code_lines,
                                              function_fullname=function.full_name, contract_name=contract.name, source_file=os.path.join(source, file_name_sc))
                            nx_g.add_edge(node.node_id, false_node.node_id, edge_type='if_false', label='False')
                            
                    else:
                        for son_node in node.sons:
                            if son_node:
                                if son_node.node_id not in nx_g.nodes():
                                    node_label, node_type, node_expression, node_irs, node_info_vulnerabilities, node_source_code_lines = get_node_info(son_node, list_vul_info_sc)
                                    nx_g.add_node(son_node.node_id, label=node_label,
                                                  node_type=node_type, node_expression=node_expression, node_irs=node_irs,
                                                  node_info_vulnerabilities=node_info_vulnerabilities,
                                                  node_source_code_lines=node_source_code_lines,
                                                  function_fullname=function.full_name, contract_name=contract.name, source_file=os.path.join(source, file_name_sc))
                                nx_g.add_edge(node.node_id, son_node.node_id, edge_type='next', label='Next')

                nx_graph = nx_g
                # add FUNCTION_NAME node
                node_function_name = file_name_sc + '_' + contract.name + '_' + function.full_name
                node_function_source_code_lines = function.source_mapping.lines
                node_function_info_vulnerabilities = get_vulnerabilities_of_node_by_source_code_line(node_function_source_code_lines, list_vul_info_sc)
                nx_graph.add_node(node_function_name, label=node_function_name,
                                  node_type='FUNCTION_NAME', node_expression=None, node_irs=None,
                                  node_info_vulnerabilities=node_function_info_vulnerabilities,
                                  node_source_code_lines=node_function_source_code_lines,
                                  function_fullname=function.full_name, contract_name=contract.name, source_file=os.path.join(source, file_name_sc))
                
                if 0 in nx_graph.nodes():
                    nx_graph.add_edge(node_function_name, 0, edge_type='next', label='Next')

                nx_graph = nx.relabel_nodes(nx_graph, lambda x: contract.name + '_' + function.full_name + '_' + str(x), copy=False)

                if merged_graph is None:
                    merged_graph = deepcopy(nx_graph)
                else:
                    merged_graph = nx.disjoint_union(merged_graph, nx_graph)

            if merge_contract_graph is None:
                merge_contract_graph = deepcopy(merged_graph)
            elif merged_graph is not None:
                merge_contract_graph = nx.disjoint_union(merge_contract_graph, merged_graph)
        
        if full_graph is None:
            full_graph = deepcopy(merge_contract_graph)
        elif merge_contract_graph is not None:
            full_graph = nx.disjoint_union(full_graph, merge_contract_graph)

    os.makedirs(os.path.dirname(compiled_ok_path), exist_ok=True)
    os.makedirs(os.path.dirname(compiled_fail_path), exist_ok=True)
    with open(compiled_ok_path, "w") as f:
        for sc in compiled_ok:
            f.write(sc + "\n")

    with open(compiled_fail_path, "w") as f:
        for sc in compiled_fail:
            f.write(sc + "\n")
    # for node, node_data in full_graph.nodes(data=True):
    #     if node_data['node_info_vulnerabilities'] is not None:
    #         print('Node has vulnerabilities:', node, node_data)
    print(f'Extract {count}/{len(smart_contracts)} sources')
    # nx.nx_agraph.write_dot(full_graph, output.replace('.gpickle', '.dot'))
    nx.write_gpickle(full_graph, output)

    
    
def load_combined_vulns(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)
from collections import Counter
import random

if __name__ == '__main__':

    output = f"/kaggle/working/out/cfg_compressed_graphs.gpickle"
    data_vulnerabilities = None
    vulnerabilities_json_files = "/kaggle/input/resourcee/graph/dataset/combined_vulnerabilities.json"
    data_vulnerabilities = load_combined_vulns(vulnerabilities_json_files)
    
    
    smart_contracts = []
    counts = Counter()
    sc_sum = f'/kaggle/input/resourcee/graph/dataset/paths.txt'
    with open(sc_sum, "r") as f:
        sc_paths = { line.strip() for line in f if line.strip() }

    smart_contracts = []
    counts = Counter()
    prefix = "home/kali/KLTN"
    smart_contracts = []
    
    

    for entry in data_vulnerabilities:
        if entry.get("source", "").lower() not in ["scrawld"]:
            continue
        rel_path = entry.get("path", "").lstrip("/")  # e.g. "dataset/.../foo.sol"
        matches = [full for full in sc_paths if full.endswith(rel_path)]
        if matches:
            for full in matches:
                if full.startswith(prefix):
                    rel_sc = full[len(prefix):].lstrip("/")
                else:
                    rel_sc = full.lstrip("./") 
              
                smart_contracts.append(rel_sc)
                counts["sbcurated"] += 1
        else:
            counts["missing"] += 1

    random.seed(42) 
    random.shuffle(smart_contracts)
    n = len(smart_contracts)
    chunk_size = n // 4
    
    part1 = smart_contracts[:chunk_size]
    part2 = smart_contracts[chunk_size:2*chunk_size]
    part3 = smart_contracts[2*chunk_size:3*chunk_size]
    part4 = smart_contracts[3*chunk_size:]
   
  
    compress_full_smart_contracts(part1, None, output, vulnerabilities=data_vulnerabilities)
    
    
    
        
   


  0%|          | 0/1998 [00:00<?, ?it/s]


/kaggle/working/artifacts/solc-0.4.25/solc-0.4.25
1
[{'category': 'Arithmetic', 'properties': 'SWC-101', 'lines': [135, 153, 174, 219, 370]}, {'category': 'Reentrancy', 'properties': 'SWC-107', 'lines': [284, 291, 299, 343, 356, 358, 369, 381, 383, 395, 409, 410, 418, 430, 443, 444, 452]}, {'category': 'Unchecked External Calls', 'properties': 'SWC-104', 'lines': [291, 336, 381, 410, 444, 623, 636]}]


  0%|          | 1/1998 [00:01<1:04:16,  1.93s/it]


/kaggle/working/artifacts/solc-0.4.25/solc-0.4.25
2
[{'category': 'Arithmetic', 'properties': 'SWC-101', 'lines': [51, 167]}]


  0%|          | 2/1998 [00:02<35:18,  1.06s/it]  

...


/kaggle/working/artifacts/solc-0.4.25/solc-0.4.25
51
[{'category': 'Arithmetic', 'properties': 'SWC-101', 'lines': [43, 279, 280, 291, 311]}]


  3%|▎         | 51/1998 [00:48<35:38,  1.10s/it]


/kaggle/working/artifacts/solc-0.4.25/solc-0.4.25


  3%|▎         | 51/1998 [00:49<31:26,  1.03it/s]


KeyboardInterrupt: 